In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import gaussian_kde

In [62]:
from scipy.stats import norm
from numpy.random import RandomState
from scipy.stats.sampling import NumericalInversePolynomial

def run_monte_carlo(
    no_of_users: int, mean_of_dist=18, std_of_dist=2, repeatable=True, domain=(0, 24)
):
    """Run Monte Carlo simulation to generate random EV plug-in time."""
    extended_domain = (0, 34)
    if repeatable:
        random_state = RandomState(42)
    else:
        random_state = RandomState()

    # assume normal distribution
    normal_dist = norm(mean_of_dist, std_of_dist)

    # using NumericalInversePolynomial to generate inverse cdf with bounds

    # if domain is (0, 24), extend bounds and remap
    if domain == (0, 24):
        domain = extended_domain
    inverse_cdf = NumericalInversePolynomial(normal_dist, domain=domain)
    inverse_cdf_samples = inverse_cdf.rvs(size=no_of_users, random_state=random_state)

    # if domain is (0, 24), remap to (0, 24)
    if domain == extended_domain:
        # anything greater than 24 is remapped to the morning
        inverse_cdf_samples[inverse_cdf_samples > 24] = inverse_cdf_samples[inverse_cdf_samples > 24] - 24

    return inverse_cdf_samples



In [95]:
mean_of_dist = 7
std_of_dist = 1.5
domain = (0, 24)
no_samples = 1000

plug_out = run_monte_carlo(no_samples, mean_of_dist, std_of_dist, domain=domain)

mean_of_dist = 18
plug_in = run_monte_carlo(no_samples, mean_of_dist, std_of_dist, domain=domain)

In [103]:
# list of lists of random samples
np.mean([np.array(np.random.normal(7, 1.5, 1000)), np.array(np.random.normal(18, 1.5, 1000))], axis=0)

array([14.13580184, 12.05154159, 12.30911428, 12.37047629, 12.53737197,
       13.15135388, 12.62904619, 12.66194855, 10.74941891, 12.74398404,
       12.46842114, 13.08830906, 12.73873221, 12.35706964, 11.86144533,
       15.45488299, 12.86290221, 11.52780355, 11.68730306, 13.08642472,
       11.84331229, 11.52967748, 11.55912957, 10.9848715 , 10.11363651,
       12.48972559, 15.18847771, 12.93868411, 11.51867612, 10.91971607,
       13.13796789, 10.89247051, 12.54614405, 12.68983381, 11.82799316,
       12.64113902, 12.29246314, 13.49326033, 12.70940812, 12.81854289,
       13.13390831, 12.5275673 , 13.13980155, 11.31335649, 12.65915167,
       12.82999628, 12.94120536, 11.04846134, 12.13870205, 12.3864647 ,
       10.4976568 , 13.3729693 , 13.00525291, 12.00777727, 13.46045171,
       11.53178321, 11.79685502, 13.19724858, 12.58569054, 12.99439373,
       12.70315682, 12.38140576, 12.23786486, 13.34534743, 12.33341698,
       12.66369378, 12.74210963, 12.90333955, 14.31566314, 11.84

In [96]:
s_plug_in = pd.Series(plug_in)
s_plug_in_count = s_plug_in.value_counts(bins=range(0, 25)).sort_index().cumsum()

s_plug_out = pd.Series(plug_out)
s_plug_out_count = s_plug_out.value_counts(bins=range(0, 25)).sort_index().cumsum()

In [100]:
# over time count the number of cars plugged in
s_plug_in_out_count = no_samples - (s_plug_out_count - s_plug_in_count)

import plotly.express as px

fig = px.bar(
    x=s_plug_in_out_count.index.mid,
    y=s_plug_in_out_count.values,
    labels={"x": "Time of day", "y": "Number of cars"},
    title="Number of cars plugged in over time",
    template="plotly_dark",
)
fig.show()

In [39]:
mean_of_dist = 0.52
std_of_dist = 0.3
domain = (0, 1)
no_samples = 1000

soc_samples_group1_1 = run_monte_carlo(no_samples, mean_of_dist=mean_of_dist, std_of_dist=std_of_dist, domain=domain, repeatable=False)
soc_samples_group1_2 = run_monte_carlo(no_samples, mean_of_dist=mean_of_dist, std_of_dist=std_of_dist, domain=domain, repeatable=False)


mean_of_dist = 0.62
std_of_dist = 0.3
soc_samples_group2_1 = run_monte_carlo(no_samples, mean_of_dist=mean_of_dist, std_of_dist=std_of_dist, domain=domain, repeatable=False)
soc_samples_group2_2 = run_monte_carlo(no_samples, mean_of_dist=mean_of_dist, std_of_dist=std_of_dist, domain=domain, repeatable=False)

In [50]:
unique_labels = set(group_labels)
colors = ["#A56CC1", "#C5B0D5", "#D8B5D5", "#E7C8D8", "#F4D9E9", "#F8E1F2"]
# one color from colors for each group
color_map = {label: colors.pop() for label in unique_labels}


group1 #F8E1F2
group2 #F4D9E9


In [51]:
import plotly.figure_factory as ff

hist_data = [soc_samples_group1_1, soc_samples_group1_2, soc_samples_group2_1, soc_samples_group2_2]
group_labels = ['group1', 'group1', 'group2', 'group2']
colors = ["#A56CC1", "#C5B0D5", "#D8B5D5", "#E7C8D8", "#F4D9E9", "#F8E1F2"]
metric = 'SOC'
color_map = {label: colors.pop() for label in set(group_labels)}
colors_mod = [color_map[label] for label in group_labels]

fig = ff.create_distplot(hist_data, group_labels, colors=colors_mod, show_hist=False, show_rug=False)
fig.update_layout(
    title_text=f"Distribution of {metric}",
    xaxis_title_text=f"{metric}",
    yaxis_title_text="Density",
)
names = set()
fig.for_each_trace(
    lambda trace:
        trace.update(showlegend=False)
        if (trace.name in names) else names.add(trace.name))
fig.show()

In [38]:
import plotly.figure_factory as ff

hist_data = [soc_samples_1, soc_samples_2]
group_labels = ['Group 1', 'Group 2']
colors = ['#333F44', '#37AA9C']

fig = ff.create_distplot(hist_data, group_labels, bin_size=.05, curve_type='normal', colors=colors, show_hist=True, show_rug=False)
fig.update_layout(
    title_text='Distribution of SOC',
    xaxis_title_text='SOC',
    yaxis_title_text='Density',
)
fig.show()


In [ ]:
# estimate PDF of plug-in-soc histogram
plug_in_soc_data = {
    2.5: 0.005,
    7.5: 0.016,
    12.5: 0.027,
    17.5: 0.04,
    22.5: 0.046,
    27.5: 0.048,
    32.5: 0.07,
    37.5: 0.045,
    42.5: 0.082,
    47.5: 0.06,
    52.5: 0.08,
    57.5: 0.085,
    62.5: 0.087,
    67.5: 0.07,
    72.5: 0.055,
    77.5: 0.058,
    82.5: 0.04,
    87.5: 0.03,
    92.5: 0.01,
    97.5: 0.005,
}

plug_in_soc_df = pd.DataFrame.from_dict(plug_in_soc_data, orient='index').reset_index()
plug_in_soc_df.columns = ['plug_in_soc', 'proportion']
mean = plug_in_soc_df['plug_in_soc'] * plug_in_soc_df['proportion']
g_kde = gaussian_kde(plug_in_soc_df['plug_in_soc'], weights=plug_in_soc_df['proportion'])
x = np.linspace(0, 100, 1000)
y = g_kde(x)

fig, ax1 = plt.subplots(figsize=(12,6))
sns.lineplot(x=x, y=y, color='red', ax=ax1)
ax2 = ax1.twinx()
sns.barplot(x='plug_in_soc', y='proportion', data=plug_in_soc_df, ax=ax2)

plt.xlabel('Plug-in SoC')
plt.ylabel('Proportion')
plt.title('Plug-in SoC Histogram')
plt.show()

In [ ]:
g_kde = gaussian_kde(plug_in_soc_df['plug_in_soc'], weights=plug_in_soc_df['proportion'])

# plot g_kde
x = np.linspace(0, 100, 1000)
y = g_kde(x)
plt.figure(figsize=(12,6))
plt.plot(x, y)
plt.xlabel('Plug-in SoC')
plt.ylabel('Proportion')
plt.title('Plug-in SoC PDF')
plt.show()

In [ ]:
sample_size = 10000
monte_carlo_sample = g_kde.resample(sample_size)
# plot
fig, ax = plt.subplots(figsize=(12,6))
sns.distplot(monte_carlo_sample, ax=ax)
plt.xlabel('Plug-in SoC')
plt.ylabel('Density')
plt.title('Plug-in SoC Monte Carlo Sample')
plt.show()

In [ ]:
monte_carlo_sample

In [ ]:
# plot g_kde
x = np.linspace(0, 100, 1000)
y = g_kde(x)
plt.plot(x, y)
plt.xlabel('Plug-in SoC')
plt.ylabel('Density')
plt.title('Plug-in SoC Density')
plt.show()

In [ ]:
# estimate PDF of plug-in soc
from scipy.stats import norm
from scipy.stats.sampling import NumericalInversePolynomial

class StandardNormal:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def pdf(self, x):
        return norm.pdf(x, loc=self.mean, scale=self.std)
    
mean = 52
std = 30
plug_in_soc_pdf = StandardNormal(mean, std)

x = np.linspace(0, 100, 1000)
y = plug_in_soc_pdf.pdf(x)

plt.plot(x, y, color='blue')
plt.xlabel('Plug-in SoC')
plt.ylabel('Probability Density')
plt.title('Plug-in SoC PDF')
plt.show()

# inverse cdf
inverse_cdf = NumericalInversePolynomial(plug_in_soc_pdf, domain=(0, 100))
# inverse_cdf_samples = inverse_cdf.rvs(size=10000)

# sns.histplot(inverse_cdf_samples, color='blue')
# plt.xlabel('Plug-in SoC')
# plt.ylabel('Proportion')
# plt.title('Plug-in SoC Histogram')
# plt.show()

In [ ]:
from numpy.random import RandomState

no_of_users = 10000
random_state = RandomState(42)
inverse_cdf_samples = inverse_cdf.rvs(size=no_of_users, random_state=random_state)

# plot
fig, ax = plt.subplots(figsize=(12,6))
sns.distplot(inverse_cdf_samples, ax=ax)
plt.xlabel('Plug-in SoC')
plt.ylabel('Density')
plt.title('Plug-in SoC Monte Carlo Sample')
plt.show()

In [ ]:
from scipy.stats import norm
from numpy.random import RandomState
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

class StandardNormal:
    """Standard normal distribution"""
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def pdf(self, x):
        return norm.pdf(x, loc=self.mean, scale=self.std)

    def cdf(self, x):
        return norm.cdf(x, loc=self.mean, scale=self.std)
    
    def inverse_cdf(self, x):
        return norm.ppf(x, loc=self.mean, scale=self.std)
    
mean = 52
std = 30
no_of_users = 10000
plug_in_soc_pdf = StandardNormal(mean, std, domain=(0, 100))

# inverse cdf
random_state = RandomState(42)
mc_samples = plug_in_soc_pdf.inverse_cdf(random_state.rand(no_of_users))

# plot
x = np.linspace(0, 100, 1000)
y = plug_in_soc_pdf.pdf(x)

plt.plot(x, y, color='blue')
plt.xlabel('Plug-in SoC')
plt.ylabel('Probability Density')
plt.title('Plug-in SoC PDF')
plt.show()

sns.histplot(mc_samples, color='blue')
plt.xlabel('Plug-in SoC')
plt.ylabel('Proportion')
plt.title('Plug-in SoC Histogram')
plt.show()